In [1]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver


Executing: /tmp/apt-key-gpghome.CSSbZGJ1iu/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.fDUML2lT51/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.6FVfiDfCZL/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://deb.debian.org/debian bust

In [2]:
pip install selenium beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 42.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.7 MB/s eta 0:00:00


In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from datetime import datetime
import re
import time

In [4]:
# ヘッドレスモードで起動するためのオプションを設定
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

# Chromeを立ち上げる
chrome_driver = webdriver.Chrome(options=chrome_options)


In [5]:
# 日経平均株価のページにアクセス
chrome_driver.get('https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month')


In [ ]:
#HTML取得
nikkeihtml = chrome_driver.page_source

In [8]:
# 株価データを抽出する関数
def extract_stock_data(nikkeihtml):
    soup = BeautifulSoup(nikkeihtml)

    stock_lists = [0,0,0,0,0]
    #情報取得
    for i in range(1,6):
        st_data = soup.select_one(f'#highcharts-0 > div.highcharts-tooltip > span > table > tbody > tr:nth-child(1) > td:nth-child({i})')
        st_data_value = st_data.text

        #文字列をdatetimeに変換
        if  i == 1:
            date = datetime.strptime(st_data_value,'%Y/%m/%d')
            st_data_value = date.strftime('%Y-%m-%d')

        stock_lists[i-1] = st_data_value

    return stock_lists


# 株価データを取得する関数。
def get_stock_values(chrome_driver):

    chrome_driver.get('https://www.nikkei.com/markets/worldidx/chart/nk225/?type=6month')

    # グラフ要素を最大で10回の試行で取得する
    for _ in range(10):
        graph = chrome_driver.find_elements(By.CSS_SELECTOR, '.highcharts-grid')[1]
        if graph is not None:
            break
        print('continue find elements')

    # マウスの動きを記述
    actions         = ActionChains(chrome_driver)

    # グラフの中心にマウスを移動
    actions.move_to_element(graph)

    # グラフの横幅（width）の半分の値だけ右にマウスをずらし、グラフの右端にマウスを移動
    actions.move_by_offset((graph.rect['width'] // 2),0)

    # 記述した動作の実行
    actions.perform()

    # グラフの横幅を取得
    graph_width     = graph.rect['width']

    stock_values    = []
    html            = chrome_driver.page_source.encode('utf-8')

    # 株価データを抽出する。
    stock_values.append(extract_stock_data(html))

        # グラフを1ピクセル左に移動しながら、株価データを抽出する
    for k in range(graph_width - 1):
        actions         = ActionChains(chrome_driver)

        actions.move_by_offset(-1, 0).perform()

        html            = chrome_driver.page_source.encode('utf-8')
        temp_value      = extract_stock_data(html)

        if temp_value not in stock_values:
            stock_values.append(temp_value)



    return stock_values

start_time = time.time()
print(f'Scraping time: {time.time() - start_time}')
result = get_stock_values(chrome_driver)

# ブラウザを閉じる
chrome_driver.quit()

# 結果を表示する
print('日付, 始値, 高値, 安値, 終値')
for data in result:
    print(', '.join(data))


Scraping time: 7.009506225585938e-05
日付, 始値, 高値, 安値, 終値
2023-10-24, 始値: 31,157.01, 高値: 31,210.26, 安値: 31,120.33, 終値: 31,125.13
2023-10-23, 始値: 31,151.98, 高値: 31,177.41, 安値: 30,974.26, 終値: 30,999.55
2023-10-20, 始値: 31,164.89, 高値: 31,428.97, 安値: 31,093.9, 終値: 31,259.36
2023-10-19, 始値: 31,579.54, 高値: 31,669.42, 安値: 31,399.17, 終値: 31,430.62
2023-10-18, 始値: 32,033.81, 高値: 32,101.47, 安値: 31,866.95, 終値: 32,042.25
2023-10-17, 始値: 32,063.79, 高値: 32,260.77, 安値: 31,901.39, 終値: 32,040.29
2023-10-16, 始値: 31,983.04, 高値: 31,999.79, 安値: 31,564.31, 終値: 31,659.03
2023-10-13, 始値: 32,328.39, 高値: 32,533.08, 安値: 32,249.03, 終値: 32,315.99
2023-10-12, 始値: 32,120.94, 高値: 32,494.66, 安値: 32,120.94, 終値: 32,494.66
2023-10-11, 始値: 31,847.1, 高値: 32,037.07, 安値: 31,804.09, 終値: 31,936.51
2023-10-10, 始値: 31,314.67, 高値: 31,818.26, 安値: 31,314.67, 終値: 31,746.53
2023-10-06, 始値: 31,003.95, 高値: 31,160.45, 安値: 30,928.16, 終値: 30,994.67
2023-10-05, 始値: 30,733.65, 高値: 31,083.9, 安値: 30,565.32, 終値: 31,075.36
2023-10-04, 始値: 30,765.0